In [1]:
import pandas as pd
import numpy as np

In [2]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

In [3]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [4]:
credits.head(1)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [5]:
# la funzionecrea una serie di dataframe, in cui ogni dataframe rappresenta una cella json (es. cast di credits) del 
# dataframe originale e restituisce un unico dataframe
def normalizza (movie_id, colonna, dataset):
    temp = dataset.set_index(movie_id)[colonna].apply(pd.read_json)
    def setta_chiave(indice):
        temp[indice][movie_id] = indice
        return temp
    dataset[movie_id].apply(setta_chiave)
    return pd.concat(temp.values.tolist())

1) For each movie, compute the number of cast members

In [6]:
cast = normalizza('movie_id', 'cast', credits)
cast_number = cast.groupby('movie_id').size()
cast_number = cast_number.to_frame().rename(columns= {0: 'cast_number'}).reset_index()
cast_number = pd.merge(left = cast_number, right = credits[['movie_id', 'title']], on = 'movie_id')
cast_number.head()

,movie_id,cast_number,title
0,5,24,Four Rooms
1,11,106,Star Wars
2,12,24,Finding Nemo
3,13,66,Forrest Gump
4,14,40,American Beauty


2) How many movies do not have a homepage?

In [7]:
moviey = movies[movies['homepage'].isnull()]
len(moviey)

3091

3) For each year, how many movies do not have a homepage?

In [8]:
moviey['year'] = movies['release_date'].str.extract('(\d\d\d\d)')
moviey = moviey.groupby('year').size().to_frame().rename(columns = {0 : 'number_movies'}).reset_index()
moviey.head()

C:\Users\aless\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  """Entry point for launching an IPython kernel.
C:\Users\aless\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,year,number_movies
0,1916,1
1,1925,1
2,1927,1
3,1929,2
4,1930,1


4) Extract the domain of each homepage.

In [9]:
import tldextract
movie_d = movies[movies['homepage'].notnull()][['id', 'original_title', 'homepage']]
# la funzione restituisce il numero di split (quindi il numero di homepage per ciascun film)
def conta(colonna):
    split = colonna.split()
    return len(split)
movie_d['split'] = movie_d['homepage'].apply(conta)
# troviamo eventuali film con più di una homepage
salva_id = movie_d[movie_d['split'] != 1][['homepage', 'id', 'original_title']]
salva_id

,homepage,id,original_title
3730,http://www.cargoderfilm.ch http://cargothemovi...,34069,Cargo


In [10]:
# creiamo un dataframe provvisorio che ha come numero di righe il numero totale delle homepage che si riferiscono agli stessi
# film (in questo caso solo il film "Cargo" con due homepage)
salva_id = salva_id.drop('original_title', axis = 1)
provvisorio = pd.DataFrame(columns = ['homepage', 'id'], index = range(len(salva_id) + 1))
# crea una lista con tutte le homepage che si riferiscono ad un film
for element in salva_id.reset_index()['index']: 
    split = salva_id['homepage'][element].split()
# salva le homepage presenti nella lista all'interno del dataframe provvisorio    
    for elemen in range(len(salva_id) + 1):
        provvisorio['homepage'][elemen] = split[elemen] 
        provvisorio['id'][elemen] = salva_id['id'][element]
provvisorio

,homepage,id
0,http://www.cargoderfilm.ch,34069
1,http://cargothemovie.com,34069


In [11]:
# viene salvato il dataframe escludendo i film con più di un homepage
movie_d1 = movie_d[movie_d['split'] == 1]
# vengono aggiunte al dataframe provvisorio tutte le colonne di movie_d
provvisorio = pd.merge(left = provvisorio, right = movie_d, how = 'left', on = 'id').drop('homepage_y', axis = 1).rename(
                                                                        columns = {'homepage_x': 'homepage'})
provvisorio

,homepage,id,original_title,split
0,http://www.cargoderfilm.ch,34069,Cargo,2
1,http://cargothemovie.com,34069,Cargo,2


In [12]:
movie_d = pd.concat([movie_d1, provvisorio])
# tldextract takes a string URL and splits it into its subdomain, domain, and suffix component
def estrai(colonna):
    return tldextract.extract(colonna).suffix
movie_d['domain'] = movie_d['homepage'].apply(estrai)
movie_d = movie_d.drop('split', axis = 1)
movie_d.head()

,homepage,id,original_title,domain
0,http://www.avatarmovie.com/,19995,Avatar,com
1,http://disney.go.com/disneypictures/pirates/,285,Pirates of the Caribbean: At World's End,com
2,http://www.sonypictures.com/movies/spectre/,206647,Spectre,com
3,http://www.thedarkknightrises.com/,49026,The Dark Knight Rises,com
4,http://movies.disney.com/john-carter,49529,John Carter,com


In [13]:
# il caso particolare del film "Cargo"
movie_d[movie_d['original_title'] == 'Cargo']

,homepage,id,original_title,domain
0,http://www.cargoderfilm.ch,34069,Cargo,ch
1,http://cargothemovie.com,34069,Cargo,com


5) Extract a set of normalized tables. That is, each entry of a normalized table must contain exactly one value (not a list or a dictionary)

In [14]:
movies_n = movies.drop(['genres', 'keywords', 'production_companies', 'production_countries', 'spoken_languages'], axis = 1)
credits_n = credits.drop(['cast', 'crew'], axis = 1)
genres = normalizza('id','genres', movies)
keywords = normalizza('id', 'keywords', movies)
production_companies = normalizza('id','production_companies', movies)
production_countries = normalizza('id', 'production_countries', movies).rename(columns = {'iso_3166_1':'country_id'})
spoken_languages = normalizza('id', 'spoken_languages', movies).rename(columns = {'iso_639_1':'language_id'})
crew = normalizza('movie_id','crew', credits)

In [15]:
def valuta_chiave_singola(df, colonna):
    return len(df[colonna]) == len(df[colonna].drop_duplicates())
def valuta_chiave_composta(df, colonna1, colonna2):
    temp = df.applymap(str)
    temp['temp'] = temp[colonna1] + ' ' + temp[colonna2]
    return len(temp['temp']) == len(temp['temp'].drop_duplicates())

In [16]:
# movies_normalizzata
movies_normalizzata = movies_n[['id', 'original_title', 'title', 'budget', 'homepage', 'original_language', 'overview', 
                            'popularity', 'release_date', 'revenue', 'runtime', 'status', 'tagline', 'vote_average', 
                            'vote_count']].rename(columns = {'id' : 'movie_id', 'title' : 'en_title'})

In [17]:
# movie_id è chiave primaria?
valuta_chiave_singola(movies_normalizzata, 'movie_id')

True

In [18]:
movies_normalizzata.head()

,movie_id,original_title,en_title,budget,homepage,original_language,overview,popularity,release_date,revenue,runtime,status,tagline,vote_average,vote_count
0,19995,Avatar,Avatar,237000000,http://www.avatarmovie.com/,en,"In the 22nd century, a paraplegic Marine is di...",150.437577,2009-12-10,2787965087,162.0,Released,Enter the World of Pandora.,7.2,11800
1,285,Pirates of the Caribbean: At World's End,Pirates of the Caribbean: At World's End,300000000,http://disney.go.com/disneypictures/pirates/,en,"Captain Barbossa, long believed to be dead, ha...",139.082615,2007-05-19,961000000,169.0,Released,"At the end of the world, the adventure begins.",6.9,4500
2,206647,Spectre,Spectre,245000000,http://www.sonypictures.com/movies/spectre/,en,A cryptic message from Bond’s past sends him o...,107.376788,2015-10-26,880674609,148.0,Released,A Plan No One Escapes,6.3,4466
3,49026,The Dark Knight Rises,The Dark Knight Rises,250000000,http://www.thedarkknightrises.com/,en,Following the death of District Attorney Harve...,112.312950,2012-07-16,1084939099,165.0,Released,The Legend Ends,7.6,9106
4,49529,John Carter,John Carter,260000000,http://movies.disney.com/john-carter,en,"John Carter is a war-weary, former military ca...",43.926995,2012-03-07,284139100,132.0,Released,"Lost in our world, found in another.",6.1,2124


In [19]:
# genres normalizzata
genres_normalizzata = genres.rename(columns = {'id' : 'movie_id', 'name' : 'genre'})

In [20]:
# movie_id e genres sono la chiave primaria?
valuta_chiave_composta(genres_normalizzata, 'movie_id', 'genre')

True

In [21]:
genres_normalizzata.head()

,movie_id,genre
0.0,19995,Action
1.0,19995,Adventure
2.0,19995,Fantasy
3.0,19995,Science Fiction
0.0,285,Adventure


In [22]:
#keywords normalizzata
keywords_normalizzata = keywords.rename(columns = {'id' : 'movie_id', 'name' : 'keyword'})

In [23]:
# movie_id e name sono la chiave primaria?
valuta_chiave_composta(keywords_normalizzata, 'movie_id', 'keyword')

True

In [24]:
keywords_normalizzata.head()

,movie_id,keyword
0.0,19995,culture clash
1.0,19995,future
2.0,19995,space war
3.0,19995,space colony
4.0,19995,society


In [25]:
# production_companies normalizzata
production_companies_normalizzata = production_companies.rename(columns = {'id' : 'movie_id', 'name' : 'production_company'})

In [26]:
# movie_id e production_company sono la chiave primaria?
valuta_chiave_composta(production_companies_normalizzata, 'movie_id', 'production_company')

True

In [27]:
production_companies_normalizzata.head()

,movie_id,production_company
0.0,19995,Ingenious Film Partners
1.0,19995,Twentieth Century Fox Film Corporation
2.0,19995,Dune Entertainment
3.0,19995,Lightstorm Entertainment
0.0,285,Walt Disney Pictures


In [28]:
# production_countries normalizzata
production_countries_normalizzata = production_countries[['id', 'country_id']].rename(columns = {'id' : 'movie_id'})

In [29]:
# movie_id e country_id sono la chiave primaria?
valuta_chiave_composta(production_countries_normalizzata, 'movie_id', 'country_id')

True

In [30]:
production_countries_normalizzata.head()

,movie_id,country_id
0.0,19995,US
1.0,19995,GB
0.0,285,US
0.0,206647,GB
1.0,206647,US


In [31]:
# countries normalizzata
countries_normalizzata = production_countries[['country_id', 'name']].drop_duplicates().rename(columns = 
                                                                                               {'name' : 'country_name'})

In [32]:
# country_id è la chiave primaria?
valuta_chiave_singola(countries_normalizzata, 'country_id')

True

In [33]:
countries_normalizzata.head()

,country_id,country_name
0.0,US,United States of America
1.0,GB,United Kingdom
0.0,JM,Jamaica
2.0,BS,Bahamas
3.0,DM,Dominica


In [34]:
# spoken_languages normalizzata
spoken_languages_normalizzata = spoken_languages[['id', 'language_id']].rename(columns = {'id' : 'movie_id'})

In [35]:
# movie_id e language_id sono la chiave primaria?
valuta_chiave_composta(spoken_languages_normalizzata, 'movie_id', 'language_id')

True

In [36]:
spoken_languages_normalizzata.head()

,movie_id,language_id
0.0,19995,en
1.0,19995,es
0.0,285,en
0.0,206647,fr
1.0,206647,en


In [37]:
# languages normalizzata
languages_normalizzata = spoken_languages[['language_id', 'name']].drop_duplicates().rename(columns = 
                                                                                               {'name' : 'language_name'})

In [38]:
# language_id è la chiave primaria?
valuta_chiave_singola(languages_normalizzata, 'language_id')

True

In [39]:
languages_normalizzata.head()

,language_id,language_name
0.0,en,English
1.0,es,Español
0.0,fr,Français
3.0,it,Italiano
4.0,de,Deutsch


In [40]:
# cast normalizzata
cast_normalizzata = cast[['credit_id', 'movie_id', 'cast_id', 'character', 'id', 'order']].rename(columns = {'credit_id' : 
                                                                                         'cast_credit_id', 'id' : 'actor_id'})

In [41]:
# cast_credit_id è la chiave primaria?
valuta_chiave_singola(cast_normalizzata, 'cast_credit_id')

True

In [42]:
cast_normalizzata.head()

,cast_credit_id,movie_id,cast_id,character,actor_id,order
0.0,5602a8a7c3a3685532001c9a,19995,242.0,Jake Sully,65731.0,0.0
1.0,52fe48009251416c750ac9cb,19995,3.0,Neytiri,8691.0,1.0
2.0,52fe48009251416c750aca39,19995,25.0,Dr. Grace Augustine,10205.0,2.0
3.0,52fe48009251416c750ac9cf,19995,4.0,Col. Quaritch,32747.0,3.0
4.0,52fe48009251416c750ac9d3,19995,5.0,Trudy Chacon,17647.0,4.0


In [43]:
# cast_actors normalizzata
cast_actors_normalizzata = cast[['id', 'name', 'gender']].drop_duplicates().rename(columns = {'id' : 'actor_id', 'name' : 
                                                                                                                 'actor_name'})

In [44]:
# actor_id è la chiave primaria?
valuta_chiave_singola(cast_actors_normalizzata, 'actor_id')

False

In [45]:
# perchè non actor_id non risulta essere la chiave primaria?
trova_errore = cast_actors_normalizzata.groupby(['actor_id', 'actor_name']).size().to_frame().rename(columns = {0 : 'size'})
trova_errore[trova_errore['size'] != 1]

,,size
actor_id,actor_name,
1189293.0,Erin Pitt,2


In [46]:
cast_actors_normalizzata = cast_actors_normalizzata.reset_index()
cast_actors_normalizzata[cast_actors_normalizzata['actor_name'] == 'Erin Pitt']

,index,actor_id,actor_name,gender
33144,10.0,1189293.0,Erin Pitt,1.0
50154,2.0,1189293.0,Erin Pitt,0.0


In [47]:
cast_actors_normalizzata = cast_actors_normalizzata.drop(50154).set_index('index')
cast_actors_normalizzata[cast_actors_normalizzata['actor_name'] == 'Erin Pitt']

,actor_id,actor_name,gender
index,,,
10.0,1189293.0,Erin Pitt,1.0


In [48]:
# ora actor_id è la chiave primaria?
valuta_chiave_singola(cast_actors_normalizzata, 'actor_id')

True

In [49]:
cast_actors_normalizzata.head()

,actor_id,actor_name,gender
index,,,
0.0,65731.0,Sam Worthington,2.0
1.0,8691.0,Zoe Saldana,1.0
2.0,10205.0,Sigourney Weaver,1.0
3.0,32747.0,Stephen Lang,2.0
4.0,17647.0,Michelle Rodriguez,1.0


In [50]:
# crew_normalizzata
crew_normalizzata = crew[['credit_id', 'movie_id', 'department', 'job', 'id']].rename(columns = {'credit_id' : 'crew_credit_id',
                                                                                                            'id' : 'crew_id'})

In [51]:
# credit_id è la chiave primaria?
valuta_chiave_singola(crew_normalizzata, 'crew_credit_id')

True

In [52]:
crew_normalizzata.head()

,crew_credit_id,movie_id,department,job,crew_id
0.0,52fe48009251416c750aca23,19995,Editing,Editor,1721.0
1.0,539c47ecc3a36810e3001f87,19995,Art,Production Design,496.0
2.0,54491c89c3a3680fb4001cf7,19995,Sound,Sound Designer,900.0
3.0,54491cb70e0a267480001bd0,19995,Sound,Supervising Sound Editor,900.0
4.0,539c4a4cc3a36810c9002101,19995,Production,Casting,1262.0


In [53]:
# crew_members normalizzata
crew_members_normalizzata = crew[['id', 'name', 'gender']].drop_duplicates().rename(columns = {'id' : 'crew_id',
                                                                                        'name' : 'member_name'})

In [54]:
# crew_id è la chiave primaria?
valuta_chiave_singola(crew_members_normalizzata, 'crew_id')

False

In [55]:
# come nel caso della tabella cast_actors_normalizzata
trova_errore = crew_members_normalizzata.groupby(['crew_id', 'member_name']).size().to_frame().rename(columns = {0 : 'size'})
trova_errore[trova_errore['size'] != 1]

,,size
crew_id,member_name,
30711.0,Kenneth Lonergan,2


In [56]:
crew_members_normalizzata = crew_members_normalizzata.reset_index()
crew_members_normalizzata[crew_members_normalizzata['member_name'] == 'Kenneth Lonergan']

,index,crew_id,member_name,gender
11921,20.0,30711.0,Kenneth Lonergan,0.0
39310,9.0,30711.0,Kenneth Lonergan,2.0


In [57]:
crew_members_normalizzata = crew_members_normalizzata.drop(11921).set_index('index')
crew_members_normalizzata[crew_members_normalizzata['member_name'] == 'Kenneth Lonergan']

,crew_id,member_name,gender
index,,,
9.0,30711.0,Kenneth Lonergan,2.0


In [58]:
# ora crew_id è la chiave primaria?
valuta_chiave_singola(crew_members_normalizzata, 'crew_id')

True

In [59]:
crew_members_normalizzata.head()

,crew_id,member_name,gender
index,,,
0.0,1721.0,Stephen E. Rivkin,0.0
1.0,496.0,Rick Carter,2.0
2.0,900.0,Christopher Boyes,0.0
4.0,1262.0,Mali Finn,1.0
5.0,1729.0,James Horner,2.0


6) For each movie, compute the gross margin (difference between revenue and budget)


In [60]:
movies_normalizzata['gross_margin'] = movies_normalizzata['revenue'] - movies_normalizzata['budget']
movies_normalizzata[['movie_id', 'en_title', 'gross_margin']].head()

,movie_id,en_title,gross_margin
0,19995,Avatar,2550965087
1,285,Pirates of the Caribbean: At World's End,661000000
2,206647,Spectre,635674609
3,49026,The Dark Knight Rises,834939099
4,49529,John Carter,24139100


7) For each movie, compute the number of crew members

In [61]:
crew_n = pd.merge(left = crew_normalizzata, right = credits_n, on = 'movie_id')
crew_number = crew_n.groupby(['movie_id', 'title']).count()['crew_credit_id'].to_frame().reset_index().rename(
                                                                                        columns = {'credit_id' : 'crew_number'})
crew_number.head()

,movie_id,title,crew_credit_id
0,5,Four Rooms,88
1,11,Star Wars,20
2,12,Finding Nemo,104
3,13,Forrest Gump,93
4,14,American Beauty,109


8) For each movie, compute the number of directors

In [62]:
directors = (crew_n[crew_n['job'] == 'Director']).groupby(['movie_id', 'title']).size().to_frame().rename(
                                                                                columns = {0 : 'number_directors'}).reset_index()
directors.head()

,movie_id,title,number_directors
0,5,Four Rooms,4
1,11,Star Wars,1
2,12,Finding Nemo,1
3,13,Forrest Gump,1
4,14,American Beauty,1


9) For each language, compute the number of movies where such language is spoken.

In [63]:
movies_language = spoken_languages.groupby(['language_id', 'name']).size().to_frame().rename(
                                                    columns = {'name' : 'language_name',0 : 'number_languages'}).reset_index()
movies_language.head()

,language_id,name,number_languages
0,af,Afrikaans,7
1,am,,1
2,ar,العربية,67
3,bg,български език,3
4,bm,Bamanankan,1


10) For each company and each decade, compute the overall revenue

In [64]:
movies_normalizzata['year'] = movies_normalizzata['release_date'].str.extract('(\d\d\d\d)')
movies_normalizzata['decade'] = (movies_normalizzata['year'].str.extract('(\d\d\d)')) + '0'
movies_companies = movies_normalizzata.merge(production_companies_normalizzata, on = 'movie_id')
overall_revenue = movies_companies.groupby(['production_company', 'decade']).sum()['revenue'].to_frame().rename(
                                                            columns = {'revenue' : 'overall_revenue'}).reset_index()
overall_revenue.head()

C:\Users\aless\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  """Entry point for launching an IPython kernel.
C:\Users\aless\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  


,production_company,decade,overall_revenue
0,"""DIA"" Productions GmbH & Co. KG",2000,44350926
1,1.85 Films,2010,98017
2,10 West Studios,2010,0
3,100 Bares,2000,33965843
4,100 Bares,2010,24000000


11) For each decade, compute the company with maximum revenue

In [65]:
maximum_revenue = movies_companies.groupby(['decade']).max()['revenue'].to_frame()
max_revenue = maximum_revenue.merge(movies_companies, on = 'revenue')
max_revenues = max_revenue[['revenue', 'decade', 'production_company']]
max_revenues.head()

,revenue,decade,production_company
0,8394751,1910,Triangle Film Corporation
1,8394751,1910,Wark Producing Corp.
2,22000000,1920,Metro-Goldwyn-Mayer (MGM)
3,400176459,1930,Selznick International Pictures
4,400176459,1930,Metro-Goldwyn-Mayer (MGM)


12) In each year, how many movies have revenue smaller than the budget?

In [66]:
movies_sm = (movies_normalizzata[movies_normalizzata['gross_margin'] < 0]).groupby('year').size().to_frame().rename(
                                                                                columns = {0 : 'number_movies'}).reset_index()
movies_sm.head()

,year,number_movies
0,1927,1
1,1947,1
2,1948,1
3,1954,1
4,1962,1


13) Distribute the revenue according to the order of appearance in a movie. Assume that the i-th actor contributes twice as much as the (i+1)-th actor to the revenue.

In [67]:
import math
cast = pd.merge(left = cast, right = movies_n[['id', 'revenue', 'title']], left_on = 'movie_id', right_on = 'id')
cast_movies = cast[['movie_id', 'revenue', 'title', 'name', 'order']]
# l'ordine non parte sempre dalla stessa cifra, infatti ad esempio il film "Avatar" parte da 0, mentre il film "Time to Choose"
# parte da 1. Inoltre l'ordine di alcuni film è discontinuo, ad esempio "El Mariachi"
cast_movies[cast_movies['title'] == 'Avatar'].head()

,movie_id,revenue,title,name,order
0,19995,2787965087,Avatar,Sam Worthington,0.0
1,19995,2787965087,Avatar,Zoe Saldana,1.0
2,19995,2787965087,Avatar,Sigourney Weaver,2.0
3,19995,2787965087,Avatar,Stephen Lang,3.0
4,19995,2787965087,Avatar,Michelle Rodriguez,4.0


In [68]:
cast_movies[cast_movies['title'] == 'Time to Choose']

,movie_id,revenue,title,name,order
93248,370662,0,Time to Choose,Oscar Isaac,1.0
93249,370662,0,Time to Choose,Jane Goodall,2.0
93250,370662,0,Time to Choose,Jerry Brown,3.0
93251,370662,0,Time to Choose,Neal Barnard,4.0
93252,370662,0,Time to Choose,Michael Pollan,5.0


In [69]:
cast_movies[cast_movies['title'] == 'El Mariachi']

,movie_id,revenue,title,name,order
106225,9367,2040920,El Mariachi,Carlos Gallardo,0.0
106226,9367,2040920,El Mariachi,Jaime de Hoyos,2.0
106227,9367,2040920,El Mariachi,Peter Marquardt,3.0
106228,9367,2040920,El Mariachi,Reinol Martinez,4.0
106229,9367,2040920,El Mariachi,Ramiro Gomez,5.0
106230,9367,2040920,El Mariachi,Consuelo Gómez,6.0
106231,9367,2040920,El Mariachi,Juan García,7.0


In [70]:
# l'ordine verrà quindi fatto partire da 0 e verrà implementato in modo continuo. 
# per spiegare i passaggi verrà preso come esempio il film "El Mariachi"
def ordina(movie_id):
    temp = cast_movies[cast_movies['movie_id'] == movie_id]
    temp = temp.reset_index().drop('index', axis = 1)
    return temp
temp = pd.Series(cast_movies['movie_id'].unique()).apply(ordina)
cast_movies = pd.concat(temp.values.tolist())
cast_movies = cast_movies.reset_index().rename(columns = {'index' : 'n_order'})
cast_movies[cast_movies['title'] == 'El Mariachi']

,n_order,movie_id,revenue,title,name,order
106225,0,9367,2040920,El Mariachi,Carlos Gallardo,0.0
106226,1,9367,2040920,El Mariachi,Jaime de Hoyos,2.0
106227,2,9367,2040920,El Mariachi,Peter Marquardt,3.0
106228,3,9367,2040920,El Mariachi,Reinol Martinez,4.0
106229,4,9367,2040920,El Mariachi,Ramiro Gomez,5.0
106230,5,9367,2040920,El Mariachi,Consuelo Gómez,6.0
106231,6,9367,2040920,El Mariachi,Juan García,7.0


In [71]:
cast_movies = cast_movies.drop(['order'], axis = 1).rename(columns = {'n_order' : 'order'})
# viene calcolato il numero totale di attori per cast
number_actor = cast_movies.groupby('movie_id').size().to_frame().reset_index().rename(columns = {0: 'number_actor'})
cast_movies = pd.merge(left = cast_movies, right = number_actor, on = 'movie_id')
# con la funzione exp viene calcolata la potenza x-esima di 2, dove x rappresenta il numero totale di attori per cast
def exp(x):
    return math.pow(2, x)
cast_movies['exp'] = cast_movies['number_actor'].apply(exp)
# per trovare la parte di revenue minore per cast, il revenue deve essere diviso per (exp - 1). Questo deriva dalla somma
# parziale x-esima - 1 della serie geometrica di ragione 2: 
# x + 2*x + (2^2)*x + ... + [2^(x-esima - 1)] = {sum_j(0)(x-esima - 1)[2^j]}*x = [(2^x-esima) - 1] * x
cast_movies['exp'] = cast_movies['exp'] - 1
cast_movies['part'] = cast_movies['revenue'] / cast_movies['exp']
cast_movies[cast_movies['title'] == 'El Mariachi']

,order,movie_id,revenue,title,name,number_actor,exp,part
106225,0,9367,2040920,El Mariachi,Carlos Gallardo,7,127.0,16070.23622
106226,1,9367,2040920,El Mariachi,Jaime de Hoyos,7,127.0,16070.23622
106227,2,9367,2040920,El Mariachi,Peter Marquardt,7,127.0,16070.23622
106228,3,9367,2040920,El Mariachi,Reinol Martinez,7,127.0,16070.23622
106229,4,9367,2040920,El Mariachi,Ramiro Gomez,7,127.0,16070.23622
106230,5,9367,2040920,El Mariachi,Consuelo Gómez,7,127.0,16070.23622
106231,6,9367,2040920,El Mariachi,Juan García,7,127.0,16070.23622


In [72]:
# viene calcolato l'ordine inverso per ogni cast, in modo che l'ordine minimo rappresenti in c_order l'ordine massimo
cast_movies['c_order'] = cast_movies['number_actor'] - cast_movies['order'] - 1
cast_movies['fattore'] = cast_movies['c_order'].apply(exp)
cast_movies['revenue_part'] = cast_movies['fattore'] * cast_movies['part']
cast_movies = cast_movies[['movie_id', 'title', 'revenue', 'name', 'order','revenue_part']]
cast_movies[cast_movies['title'] == 'El Mariachi']

,movie_id,title,revenue,name,order,revenue_part
106225,9367,El Mariachi,2040920,Carlos Gallardo,0,1.028495e+06
106226,9367,El Mariachi,2040920,Jaime de Hoyos,1,5.142476e+05
106227,9367,El Mariachi,2040920,Peter Marquardt,2,2.571238e+05
106228,9367,El Mariachi,2040920,Reinol Martinez,3,1.285619e+05
106229,9367,El Mariachi,2040920,Ramiro Gomez,4,6.428094e+04
106230,9367,El Mariachi,2040920,Consuelo Gómez,5,3.214047e+04
106231,9367,El Mariachi,2040920,Juan García,6,1.607024e+04


In [73]:
# per verificare la correttezza dei calcoli viene eseguita la somma delle revenue_part per ciascun cast. 
# inoltre si rendono revenue e revenue part in forma esplicita
somma = cast_movies.groupby('movie_id').sum()['revenue_part'].reset_index().rename(columns = {'revenue_part' : 'revenue_sum'})
cast_movies = pd.merge(left = cast_movies, right = somma, on = 'movie_id')
cast_movies['revenue_part'] = cast_movies['revenue_part'].apply(int)
cast_movies['revenue_sum'] = cast_movies['revenue_sum'].apply(int)
cast_movies[cast_movies['title'] == 'El Mariachi']

,movie_id,title,revenue,name,order,revenue_part,revenue_sum
106225,9367,El Mariachi,2040920,Carlos Gallardo,0,1028495,2040920
106226,9367,El Mariachi,2040920,Jaime de Hoyos,1,514247,2040920
106227,9367,El Mariachi,2040920,Peter Marquardt,2,257123,2040920
106228,9367,El Mariachi,2040920,Reinol Martinez,3,128561,2040920
106229,9367,El Mariachi,2040920,Ramiro Gomez,4,64280,2040920
106230,9367,El Mariachi,2040920,Consuelo Gómez,5,32140,2040920
106231,9367,El Mariachi,2040920,Juan García,6,16070,2040920


In [74]:
# inoltre per dimostrare che non ci sia nessun errore nel calcolo (tranne l'approssimazione di 1) viene calcolato il test
test = pd.DataFrame()
test['sum'] = cast_movies['revenue'] - cast_movies['revenue_sum']
test[(test['sum'] < -1) | (test['sum'] > 1)]

,sum


14) For each actor find the total revenue attributed to him/her.

In [75]:
actor_revenue = cast_movies.groupby('name').sum()['revenue_part'].to_frame().rename(columns = {'revenue_part' :
                                                                                              'total_revenue'}).reset_index()
actor_revenue.head()

,name,total_revenue
0,Jorge de los Reyes,0
1,Larry Mullen Jr.,1515389
2,"""Weird Al"" Yankovic",2503
3,'Snub' Pollard,219
4,'Wild Bill' Laczko,4


15) Find the actor that is responsible for the most overall revenue.

In [76]:
actor_revenue[actor_revenue['total_revenue'] == np.max(actor_revenue['total_revenue'])]

,name,total_revenue
50772,Tom Cruise,3976119747
